In [21]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import matplotlib.pyplot as plt
import base64
import datetime

# Imports the object in the CRUD module
from aac_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

# Authentication
username = "aacuser"
password = "myAAC"
shelter = AnimalShelter(username, password)

# Uses read_all method and creates a global dataframe
df = pd.DataFrame.from_records(shelter.read_all({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Grazioso Salvare’s logo
image_filename = './Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# Layout

app.layout = html.Div([
    
    # Unique identifier with name and current date
    html.B(html.H5('Author: Joel Gomez')),
    html.H5(datetime.datetime.now().strftime('%b. %d, %Y')), # uses datetime method

    # Grazioso Salvare Logo will also be used as a link to acces the SNHU home page
    html.A( # uses anchor element
        href = 'https://www.snhu.edu', # creates a hyperlink to acces the SNHU webpage
        children = [
            html.Center(html.Img(alt = 'Click here to enter the home page', # alternative message
                                 src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                 style = {'width' : '20%'})) # reduce size of logo
        ]
    ),

    # Main title
    html.Center(html.B(html.H1('Austin Animal Center'))),
    html.Hr(),
    
    # Buttons to be used for interactivity
    html.Div(id = 'filter-type',
             className = 'row',
             style = {'display' : 'flex'},
             children = [
                 html.Button(id = 'button_w', n_clicks_timestamp = 0, children = 'Water Rescue'),
                 html.Button(id = 'button_m', n_clicks_timestamp = 0, children = 'Mountain/Wilderness Rescue'),
                 html.Button(id = 'button_d', n_clicks_timestamp = 0, children = 'Disaster Rescue/Individual Tracking'),
                 html.Button(id = 'button_r', n_clicks_timestamp = 0, children = 'Reset')
             ]     
    ),
    
    # Data Table
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        
        # Set cell alignment to be left justified
        style_cell = {'textAlign' : 'left'},
        
        # Table header properties
        style_header = {
            'backgroundColor' : 'rgb(220,220,220)',
            'fontWeight' : 'bold',
            'minWidth' : '150px',
            'width' : '180px',
            'maxWidth' : '180px'
        },

        # Additional properties
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "single",
        column_selectable = True,
        row_selectable = "single",
        row_deletable = False,
        selected_columns = [],
        selected_rows = [],
        page_action = "native",
        page_current = 0,
        page_size = 10,        
        
    ),
    html.Br(),
    html.Hr(),
    
    # Sets pie chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            # First map to load will not point to any location
            children = dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10,
                             children=[
                                    dl.TileLayer(id="base-layer-id")]),
            
            className='col s12 m6',
            )
        ])
])


#############################################
# Interaction Between Components / Controller
#############################################


# Filters the data table according to queries  
@app.callback(
     Output('datatable-id', "data"),
    [Input('button_w', "n_clicks_timestamp"), 
     Input('button_m', "n_clicks_timestamp"), 
     Input('button_d', "n_clicks_timestamp"),
     Input('button_r', "n_clicks_timestamp")])

def select_button(b_w, b_m, b_d, b_r):
    bw = int(b_w)
    bm = int(b_m)
    bd = int(b_d)
    br = int(b_r)
    # Depending on the higher timestamp, a query is selected and the visible data updated
    if br == max(bw, bm, bd, br):
        dff = df    
    elif bw == max(bw, bm, bd, br):
        dff = pd.DataFrame.from_records(shelter.read_all({'breed':{'$in':['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                                                         'sex_upon_outcome': 'Intact Female', 
                                                         'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}))
    elif bm == max(bw, bm, bd, br):
        dff = pd.DataFrame.from_records(shelter.read_all({'breed':{'$in':['German Shepherd','Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky','Rottweiler']},
                                                         'sex_upon_outcome': 'Intact Male', 
                                                         'age_upon_outcome_in_weeks':{'$gte':26, '$lte':156}}))   
    elif bd == max(bw, bm, bd, br):
        dff = pd.DataFrame.from_records(shelter.read_all({'breed':{'$in':['Doberman Pinscher','German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
                                                         'sex_upon_outcome': 'Intact Male', 
                                                         'age_upon_outcome_in_weeks':{'$gte':20, '$lte':300}}))

    return dff.to_dict('records')


#This callback will highlight a row or a column on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'), Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_styles(selected_columns, selected_rows):
    return (
            [{
                # Sets colors despite selected rows or columns
                'if' : {'row_index':'even'}, 
                'backgroundColor' : 'rgb(220,220,220)'
            }]
        +
            [{
                # Sets color for selected columns
                'if': { 'column_id': i },
                'background_color': 'rgb(250,150,150)'} for i in selected_columns
            ] 
        +
            [{
                # Sets color for selected rows
                'if': { 'row_index': j },
                'background_color': 'rgb(250,250,200)'} for j in selected_rows
            ])

# This callback will create a pie chart based on the available data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "data")])
def update_graphs(data):
    # retieves current data, even if it is filtered
    dff = pd.DataFrame.from_dict(data)
    # If the main data remains unfiltered, the graph will be made based on animal type, for better visualization
    if dff.size == df.size:
        graph = [dcc.Graph(            
                    figure = px.pie(dff, dff['animal_type'], hole = .3, title = 'All Animal Types')
                )]
    # Otherwise, the graph will consider all available breeds
    else:
        graph = [dcc.Graph(            
                    figure = px.pie(dff, dff['breed'], hole = .3, title = 'Available Breeds')
                )]
    return graph


# This callback will create an interactive map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'selected_rows'),
     Input('datatable-id', 'data')])

def update_map(row, data):
    # Retrieves available data
    dff = pd.DataFrame.from_dict(data)
    return [
            # Austin TX is at [30.75,-97.48]
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[row[0], 'location_lat'],dff.loc[row[0], 'location_long']], children=[    
                #dl.Tooltip(dff['breed']),
                dl.Tooltip(dff.loc[row[0], 'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    #html.P(dff.loc[0,'name'])
                    html.P(dff.loc[row[0],'name'])
                ])
            ])
        ])
    ]



app